Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Sample Tutorial for Training & Deploying Image Classification Model using module twin update
* Train using custom dataset to detect soda pepsi/coke cans
* Convert trained model to IR -> Blob using Intel open vino toolkit 
* Deploy Model to the devkit using module twin update method

In [ ]:
# Setting up cloud environment
!pip install azureml-core azureml-contrib-iot azure-mgmt-containerregistry
!az extension add --name azure-cli-iot-ext

In [ ]:
import os
print(os.__file__)
#Import python library

In [ ]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

In [ ]:
### Create a Workspace, if it does not exist
#### Update the values for your ML workspace below
from azureml.core import Workspace
ws=Workspace.create(subscription_id='<subscriptionid>',
                resource_group='<Resourcegroup>',
                name='<MLWorkspace>',
                location='<Location>')
                
ws.write_config()

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Create Experiment
Experiment is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
experiment_name = 'soda_cans'

from azureml.core import Experiment
exp = Experiment(workspace = ws, name = experiment_name)

## Get data
### Option 1: Upload data files into datastore
Every workspace comes with a default datastore (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and access it from the compute target.

In [ ]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
data_path = experiment_name + '_training_data'
ds.upload(src_dir='data/soda_cans', target_path=data_path, overwrite=True)

### Option 2: Use existing datastore in Azure blob storage

from azureml.core.datastore import Datastore
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                         datastore_name='xxx', 
                                         container_name='xxx',
                                         account_name='xxxx', 
                                         account_key='xxx',
                                         create_if_not_exists=False)
data_path = "soda_cans_training_data" # This is the path to the folder in the blob container. Set this to None to get all the contents.
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

### Configure for using ACI

Linux-based ACI is available in West US, East US, West Europe, North Europe, West US 2, Southeast Asia, Australia East, East US 2, and Central US regions.  See details [here](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-quotas#region-availability)

In [ ]:
#Represents configuration for experiment that runs targeting different compute targets in Azure Machine Learning. The RunConfiguration object encapsulates the information necessary to submit a training run in an experiment.
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore=data_path, 
                   mode='download', # download files from datastore to compute target
                   overwrite=True)


Set the system to build a conda environment based on the run configuration. Once the environment is built, and if you don't change your dependencies, it will be reused in subsequent runs.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3', max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

# To create a GPU cluster, run the cell below. Note that your subscription must have sufficient quota for GPU VMs or the command will fail. To increase quota, see these instructions.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "gpucluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                           min_nodes=0,
                                                           max_nodes=4)
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)

In [ ]:
#setting up dependencies
from azureml.core.runconfig import RunConfiguration, DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_pip_package("tensorflow==1.8.0")
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("keras")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

with open("myenv.yml", "r") as f:
    print(f.read())
    
# create a new runconfig object
run_config = RunConfiguration(framework = "python")

# Set compute target
run_config.target = compute_target.name

# set the data reference of the run configuration
run_config.data_references = {ds.name: dr}

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared) -- deprecated
#run_config.auto_prepare_environment = True

# specify CondaDependencies obj
#run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow==1.8.0'])
run_config.environment.python.conda_dependencies = myenv

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the conda_dependencies.yml file on top of the base image, create a container and then execute the script in the container.

In [ ]:

from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = './02-mobilenet-transfer-learning-scripts', script = 'retrain.py', run_config = run_config, 
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--image_dir', str(ds.as_download()),
                                 '--architecture', 'mobilenet_1.0_224',
                                 '--output_graph', 'outputs/retrained_graph.pb',
                                 '--output_labels', 'outputs/output_labels.txt',
                                 '--model_download_url', 'https://raw.githubusercontent.com/rakelkar/models/master/model_output/',
                                 '--model_file_name', 'imagenet_2_frozen.pb'
                                ])
run = exp.submit(config=src)


In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

## Get the trained model

In [ ]:
trained_model_path = "outputs"

In [ ]:
# Download the retrained model and the labels locally
run.download_file(name = 'outputs/retrained_graph.pb', output_file_path = trained_model_path)
run.download_file(name = 'outputs/labels.txt', output_file_path = trained_model_path)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = trained_model_path,
                      model_name = "soda_cans",
                      tags = {"data": "Imagenet", "model": "object_detection", "type": "imagenet"},
                      description = "Retrained soda cans based on MobileNet",
                      workspace = ws)

In [ ]:
'''
### Convert the trained model to IR -> Blob using Intel Openvino toolkit for running on Devkit running Myriadx chipset
1) Setup Intel openvino toolkit on local machine https://docs.openvinotoolkit.org/latest/openvino_docs_install_guides_installing_openvino_windows.html
2) Download the retrained_graph.pb, config.json and labels.txt from the notebook vm /outputs folder to local machine
3) Model conversion to IR and Blob. Run the command from the command prompt as administrator
python "c:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_ty.py" --input_model C:\ICModels\retrained_graph.pb --input_shape (1,224,224,3) --data_type FP16
    
"C:\Program Files (x86)\IntelSWTools\openvino_2020.3.194\deployment_tools\inference_engine\bin\intel64\Release\myriad_compile.exe" -m retrained_graph.xml -ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 8 -VPU_NUMBER_OF_CMX_SLICES 8 -o mobilenetv1.blob -op FP32

4) zip mobilenetv1.blob, labels.txt and config.json to mobilenetv1.zip
5) Upload the zip to /ouput1 folder in notebook vm
6) Next step is to push the model to devkit using module twin update method

'''

In [ ]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
data_path = 'Modelpath'
ds.upload(src_dir='./output1', target_path=data_path, overwrite=True)

In [ ]:
#Generated Saas url for module twin update
from azure.storage.blob.baseblobservice import BaseBlobService,BlobPermissions
#from azure.storage.blob import BlobPermissions
from datetime import datetime, timedelta

AZURE_ACC_NAME = ds.account_name
AZURE_PRIMARY_KEY = ds.account_key
AZURE_CONTAINER = ds.container_name
AZURE_BLOB=ds.name
AZURE_File=data_path+'/mobilenetv1.zip'

service = BaseBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
sas_url  = service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))
#sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))

#block_blob_service = BlockBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
#sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))
downloadurl ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url
print('https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url)
print(sas_url)

In [ ]:
#Perform Module twin update
#Perform Module twin update
#Incorporate the connection string, device_id and the module_id values from your IoTHub

!pip install azure-iot-hub
import sys
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties

#Incorporate Iothub connection string and the default module name
#Go to Https://portal.azure.com
#Select your IoTHub
#Click on Shared access policies
#click service on right
#Copy the iothub connection string primary key

CONNECTION_STRING = "<IOTHUB connection string>"
DEVICE_ID = '<DeviceID>'
MODULE_ID = "azureeyemodule"

try:
    # RegistryManager
    iothub_registry_manager = IoTHubRegistryManager(CONNECTION_STRING)

    module_twin = iothub_registry_manager.get_module_twin(DEVICE_ID, MODULE_ID)
    print ( "" )
    print ( "Module twin properties before update    :" )
    print ( "{0}".format(module_twin.properties) )

    # Update twin
    twin_patch = Twin()
    twin_patch.properties = TwinProperties(desired={"ModelZipUrl": downloadurl})
    updated_module_twin = iothub_registry_manager.update_module_twin(
        DEVICE_ID, MODULE_ID, twin_patch, module_twin.etag
    )
    print ( "" )
    print ( "Module twin properties after update     :" )
    print ( "{0}".format(updated_module_twin.properties) )

except Exception as ex:
    print ( "Unexpected error {0}".format(ex) )
except KeyboardInterrupt:
    print ( "IoTHubRegistryManager sample stopped" )

In [ ]:
# The trained model will get pushed to the IoT Edge device via module twin update method
# Check model inferencing by connecting monitor to the devkit or by installing VLC media player : 
#Install VLC from https://www.videolan.org/vlc/ and install on “Windows” to check the camera function of “Azure Eye”.

#Check video stream:
#1.	Select Media -> Open Network Stream…
#2.	Input the network stream: “rtsp://[ip of PE-101]:8554/result” then click “Play” button.

In [ ]:
# delete cpu compute
"""
mycompute = AmlCompute(workspace=ws, name='cpucluster1')
mycompute.delete()

# delete gpu compute
mycompute = AmlCompute(workspace=ws, name='gpucluster1')
mycompute.delete()
"""

In [ ]:
# delete workspace
#ws.delete(delete_dependent_resources=True)